In [55]:
import numpy as np
import os
path = 'data/data_20_800.npy'
path = 'data/data_100_200.npy'
path = 'data/data_200_200.npy'
path = 'data/data_500_200.npy'
with open(path, 'rb') as f:
    data = np.load(f, allow_pickle=True)
len(data)

200

In [11]:
def calculate_tardiness(schedule, processing_times, due_dates):
    current_time = 0
    tardiness = 0
    for job in schedule:
        current_time += processing_times[job]
        tardiness += max(0, current_time - due_dates[job])
    return tardiness

def final_tardiness(p,d,seq):
    p = np.array(p)
    d = np.array(d)

    tardiness = np.cumsum(p[seq])-d[seq]
    tardiness = tardiness[tardiness>0]
    
    total_tardiness = tardiness.sum()

    return total_tardiness

In [12]:
def EDD(p, d):
    assigned_jobs = np.argsort(d)
    return assigned_jobs

In [13]:
def MDD_Challenger(processing_times: np.ndarray, due_dates: np.ndarray) -> np.ndarray:
    """ Find mathematical heuristic function skeleton for the single machine scheduling problem. Each job is assigned to the machine exactly once. Do not manipulate original due dates or processing times.

    Args:
        processing_times: A numpy array representing processing times of jobs.
        due_dates: A numpy array representing due dates of jobs.

    Return:
        A numpy array representing indices of jobs assigned in order as the result of applying the mathematical function to the inputs.
    """
    """Ultimate version of the job assignment function for the single machine scheduling problem."""
    current_time = 0
    schedule = []
    unscheduled_jobs = np.arange(len(processing_times))
    
    sort_order = np.lexsort((processing_times[unscheduled_jobs], due_dates[unscheduled_jobs]))
    unscheduled_jobs = unscheduled_jobs[sort_order]
    
    while unscheduled_jobs.size > 0:
        urgencies = np.maximum(processing_times[unscheduled_jobs] * 1.1 + current_time, due_dates[unscheduled_jobs])
        urgency_multiplier = (processing_times[unscheduled_jobs] / (current_time + np.max(processing_times[unscheduled_jobs])))
        urgency_multiplier[urgency_multiplier > 1] = 1
        urgency_bonus = (urgency_multiplier ** 2) / (1 + urgency_multiplier ** 2)
        urgencies *= (1 + urgency_bonus)
        impact_on_current_time = processing_times[unscheduled_jobs] / (current_time + np.sum(processing_times[unscheduled_jobs]) / len(unscheduled_jobs))
        urgencies += impact_on_current_time
        next_job = unscheduled_jobs[np.argmin(urgencies)]
        schedule.append(next_job)
        current_time += processing_times[next_job]
        unscheduled_jobs = np.delete(unscheduled_jobs, np.where(unscheduled_jobs == next_job))

        #search
        '''best_tardiness = calculate_tardiness(schedule, p, d)
        best_schedule = schedule
        for i in range(len(schedule) - 1):
            trial_schedule = schedule[:i] + schedule[i+1:] + [schedule[i]]
            trial_tardiness = calculate_tardiness(trial_schedule, p, d)
            if trial_tardiness < best_tardiness:
                best_tardiness = trial_tardiness
                best_schedule = trial_schedule[:]
        schedule = best_schedule'''
    
    return np.array(schedule)

In [14]:
def EDD_Challenger(processing_times: np.ndarray, due_dates: np.ndarray) -> np.ndarray:
    """ Find mathematical heuristic function skeleton for the single machine scheduling problem. Each job is assigned to the machine exactly once. Do not manipulate original due dates or processing times.

    Args:
        processing_times: A numpy array representing processing times of jobs.
        due_dates: A numpy array representing due dates of jobs.

    Return:
        A numpy array representing indices of jobs assigned in order as the result of applying the mathematical function to the inputs.
    """
    """Improved version of `assignment_v1` with a more enhanced penalty factor for longer processing times."""
    
    min_time = min(processing_times.min(), due_dates.min())
    max_time = max(processing_times.max(), due_dates.max())
    
    normalized_times = (processing_times - min_time) / (max_time - min_time)
    normalized_dates = (due_dates - min_time) / (max_time - min_time)
    
    relative_deviations = np.maximum(normalized_dates - normalized_times, 0)
    
    scale_factor, scale_factor_dev = 0.1, 0.3
    normalized_times_sq, normalized_times_cu = normalized_times ** 2, normalized_times ** 3
    
    penalty_times = np.where(normalized_times > 0.6, normalized_times * 12 * (1 + normalized_times_sq), 0)
    
    priority_scores = (1 - scale_factor - scale_factor_dev) * normalized_times + scale_factor * relative_deviations + scale_factor * normalized_times_sq + normalized_times_cu + penalty_times
    
    indices = np.argsort(priority_scores)
    
    return indices

In [15]:
def MDD(p, d):
    # Initialize current time
    current_time = 0

    # Create an array to track the scheduling order
    schedule = []

    # Number of jobs
    num_jobs = len(p)

    # Create an index array for tracking unscheduled jobs
    unscheduled_jobs = np.arange(num_jobs)

    # While there are unscheduled jobs
    while unscheduled_jobs.size > 0:
        # Calculate urgencies for each unscheduled job
        #urgencies = np.maximum(p[unscheduled_jobs], d[unscheduled_jobs] - current_time)
        urgencies = np.maximum(p[unscheduled_jobs]+ current_time, d[unscheduled_jobs] )
        # Find the job with the minimum urgency
        index_min_urgency = np.argmin(urgencies)
        job_to_schedule = unscheduled_jobs[index_min_urgency]
        
        # Append job id (index + 1 for 1-based id) to schedule
        schedule.append(job_to_schedule)
        
        # Update current time
        current_time += p[job_to_schedule]
        
        # Remove the job from the list of unscheduled jobs
        unscheduled_jobs = np.delete(unscheduled_jobs, index_min_urgency)
    return schedule

In [16]:
def pannerselvam(processing_times, due_dates):
    num_jobs = len(processing_times)
    # Step 1: Initialization
    index = [0] * num_jobs  # Tracks whether a job is scheduled (1) or not (0)
    final_sequence = []  # The final sequence of jobs
    L = 0  # Position in the final sequence

    while L < num_jobs:
        # Step 2: Calculate the sum of processing times of unscheduled jobs
        SUMT = sum(processing_times[i] for i in range(num_jobs) if index[i] == 0)

        # Step 3 & 4: Calculate slack and slack per unit processing time for unscheduled jobs
        USi = []
        for i in range(num_jobs):
            if index[i] == 0:
                Si = SUMT - due_dates[i]
                USi.append((Si / processing_times[i], i))

        # Step 5: Find the job with the maximum USi
        max_USi, job_k = max(USi)  # Get the job with maximum USi

        # Step 6 & 7: Update the final sequence and mark job K as scheduled
        L += 1
        final_sequence.append(job_k)
        index[job_k] = 1

    # Step 9: Calculate completion times and tardiness
    completion_times = []
    total_tardiness = 0
    current_time = 0
    for job in final_sequence:
        current_time += processing_times[job]
        completion_times.append(current_time)
        tardiness = max(0, completion_times[-1] - due_dates[job])
        total_tardiness += tardiness

    # printing
    #print("Final sequence:", [job + 1 for job in final_sequence])  # Adjusting for 1-indexing
    #print("Total tardiness:", total_tardiness)

    return final_sequence, total_tardiness



In [17]:
import numpy as np
import warnings
def edd_sort(p, d):
    sorted_indices = np.lexsort((d, p))
    p = p[sorted_indices]
    d = d[sorted_indices]
    return p, d

def schedule_it(U, S, job):
    if job in U:
        index_to_delete = np.where(U == job)[0]
        U = np.delete(U, index_to_delete)
        S = np.append(S, job)
    else: 
        warnings.warn("logical error")
    return U, S

def calculate_tardiness(schedule, processing_times, due_dates):
    current_time = 0
    tardiness = 0
    for job in schedule:
        current_time += processing_times[job]
        tardiness += max(0, current_time - due_dates[job])
    return tardiness

def final_tardiness(p,d,seq):
    p = np.array(p)
    d = np.array(d)

    tardiness = np.cumsum(p[seq])-d[seq]
    tardiness = tardiness[tardiness>0]
    
    return tardiness.sum()

def PSK(p, d):
    p, d= edd_sort(p, d)
    U = np.arange(len(d))# Unscheduled jobs-indices after sorting
    S = np.array([])  # Scheduled jobs
    C = 0  # Completion time
    while len(U) > 0:
        job_i = U[0]
        i_ind = 0
        #job_i_ind = 0 #first job in U
        #two straightforward assignment steps
        if len(U) == 1: #step 1
            U, S = schedule_it(U, S, job_i) #schedule i
            break #end scheduling
        if C + p[job_i] >= d[job_i]: #step 2
            U, S = schedule_it(U, S, job_i)
            C += p[job_i]
            continue #continue the scheduling

        for j in range(i_ind+1, len(U)): #select j -next job in U
            #job_j = np.where(U == job_i)[0] + 1 #step 3
            job_j = U[j]

            if d[job_i]<= C + p[job_j]: #step 4
                U, S = schedule_it(U, S, job_i)
                C += p[job_i]
                break

            if d[job_i] <= d[job_j]: #step 5
                if U[-1] == job_j: #is j last job in U?
                    U, S = schedule_it(U, S, job_i)
                    C += p[job_i]
                    break
                else: # it is not
                    continue #select new j
            else:
                #step 6
                job_i = job_j # job_j becomes the active job now 
                if U[-1] == job_i:
                    C += p[job_i]
                    U, S = schedule_it(U, S, job_i)
                    break
                else:
                    continue
    #step 9
    S = S.astype(int) 
    tardiness = calculate_tardiness(S, p, d)

    return S, tardiness

In [61]:
opt_soln = [] 
tardiness_edd = []
tardiness_challenger = []
tardiness_mdd_chal = []
tardiness_mdd = []
tardiness_panners = []
tardiness_psk = []
for i in range(len(data)):
    o = data[i]['optimal_solution']
    opt_soln.append(o)
    p = data[i]["processing_times"].copy()
    d = data[i]["due_dates"].copy()
    
    #function call
    edd_schedule = EDD(p, d)
    mddchallenger_schedule = MDD_Challenger(p, d)
    challenger4_schedule = EDD_Challenger(p, d)
    mdd_schedule = MDD(p, d)
    panner_schedule, tt = pannerselvam(p, d)
    psk_schedule, tardiness__ = PSK(p, d)
    
    #tardiness
    temp_edd = final_tardiness(p,d,edd_schedule)
    temp_mdd_chal = final_tardiness(p, d, mddchallenger_schedule)
    temp_chal = final_tardiness(p, d, challenger4_schedule)
    temp_mdd = final_tardiness(p, d, mdd_schedule)
    temp_panner = final_tardiness(p, d, panner_schedule)
    
    #append to list
    tardiness_edd.append(temp_edd)
    tardiness_mdd_chal.append(temp_mdd_chal)
    tardiness_challenger.append(temp_chal)
    tardiness_mdd.append(temp_mdd)
    #tardiness_panners.append(tt)
    tardiness_panners.append(temp_panner)
    tardiness_psk.append(tardiness__)

In [62]:
chal_optgap = (sum(opt_soln)- sum(tardiness_challenger))/sum(opt_soln)
mddchal_optgap = (sum(opt_soln)- sum(tardiness_mdd_chal))/sum(opt_soln)
edd_optgap = (sum(opt_soln)- sum(tardiness_edd))/sum(opt_soln)
mdd_optgap = (sum(opt_soln)- sum(tardiness_mdd))/sum(opt_soln)
psk_optgap = (sum(opt_soln)- sum(tardiness_psk))/sum(opt_soln)
panner_optgap = (sum(opt_soln)- sum(tardiness_panners))/sum(opt_soln)
print('edd optimality gap: {}'.format(-edd_optgap))
print('edd challenger optimality gap: {}'.format(-chal_optgap))
print('mdd optimality gap: {}'.format(-mdd_optgap))
print('mdd challenger optimality gap: {}'.format(-mddchal_optgap))
print('psk optimality gap: {}'.format(-psk_optgap))
print('pannerselvam optimality gap: {}'.format(-panner_optgap))

print('optimal value: {}'.format(sum(opt_soln)/200))

edd optimality gap: 0.7040431139655268
edd challenger optimality gap: 0.6205434881517703
mdd optimality gap: 0.013354593754150063
mdd challenger optimality gap: 0.012927649691082002
psk optimality gap: 0.013308375901529751
pannerselvam optimality gap: 0.4897696830162503
optimal value: 798933.7


In [56]:
#R = 0.2 T = 0.2
opt_soln = [] 
tardiness_edd = []
tardiness_challenger = []
tardiness_mdd_chal = []
tardiness_mdd = []
tardiness_panners = []
tardiness_psk = []
for i in range(0, 10):
    o = data[i]['optimal_solution']
    opt_soln.append(o)
    p = data[i]["processing_times"].copy()
    d = data[i]["due_dates"].copy()
    
    #function call
    edd_schedule = EDD(p, d)
    mddchallenger_schedule = MDD_Challenger(p, d)
    challenger4_schedule = EDD_Challenger(p, d)
    mdd_schedule = MDD(p, d)
    panner_schedule, tt = pannerselvam(p, d)
    psk_schedule, tardiness__ = PSK(p, d)
    
    #tardiness
    temp_edd = final_tardiness(p,d,edd_schedule)
    temp_mdd_chal = final_tardiness(p, d, mddchallenger_schedule)
    temp_chal = final_tardiness(p, d, challenger4_schedule)
    temp_mdd = final_tardiness(p, d, mdd_schedule)
    temp_panner = final_tardiness(p, d, panner_schedule)
    
    #append to list
    tardiness_edd.append(temp_edd)
    tardiness_mdd_chal.append(temp_mdd_chal)
    tardiness_challenger.append(temp_chal)
    tardiness_mdd.append(temp_mdd)
    #tardiness_panners.append(tt)
    tardiness_panners.append(temp_panner)
    tardiness_psk.append(tardiness__)
chal_optgap = (sum(opt_soln)- sum(tardiness_challenger))/sum(opt_soln)
mddchal_optgap = (sum(opt_soln)- sum(tardiness_mdd_chal))/sum(opt_soln)
edd_optgap = (sum(opt_soln)- sum(tardiness_edd))/sum(opt_soln)
mdd_optgap = (sum(opt_soln)- sum(tardiness_mdd))/sum(opt_soln)
psk_optgap = (sum(opt_soln)- sum(tardiness_psk))/sum(opt_soln)
panner_optgap = (sum(opt_soln)- sum(tardiness_panners))/sum(opt_soln)
print('edd optimality gap for 0.2, 0.2: {}'.format(-edd_optgap))
print('edd challenger optimality gap for 0.2, 0.2: {}'.format(-chal_optgap))
print('mdd optimality gap for 0.2, 0.2: {}'.format(-mdd_optgap))
print('mdd challenger optimality gap for 0.2, 0.2: {}'.format(-mddchal_optgap))
print('psk optimality gap for 0.2, 0.2: {}'.format(-psk_optgap))
print('pannerselvam optimality gap for 0.2, 0.2: {}'.format(-panner_optgap))
print('optimal value for 0.2, 0.2: {}'.format(sum(opt_soln)/10)) # 10 for 200-instance datasets, 40 for 800-instance datasets

edd optimality gap for 0.2, 0.2: 0.6866895850882622
edd challenger optimality gap for 0.2, 0.2: 0.28281346046779243
mdd optimality gap for 0.2, 0.2: 0.07632206936375509
mdd challenger optimality gap for 0.2, 0.2: 0.06991636362883748
psk optimality gap for 0.2, 0.2: 0.07627860577072333
pannerselvam optimality gap for 0.2, 0.2: 4.2778958653704855
optimal value for 0.2, 0.2: 48316.3


In [57]:
#R = 0.2 T = 0.6
opt_soln = [] 
tardiness_edd = []
tardiness_challenger = []
tardiness_mdd_chal = []
tardiness_mdd = []
tardiness_panners = []
tardiness_psk = []
#for i in range(80,120):
for i in range(20, 30):
    o = data[i]['optimal_solution']
    opt_soln.append(o)
    p = data[i]["processing_times"].copy()
    d = data[i]["due_dates"].copy()
    
    #function call
    edd_schedule = EDD(p, d)
    mddchallenger_schedule = MDD_Challenger(p, d)
    challenger4_schedule = EDD_Challenger(p, d)
    mdd_schedule = MDD(p, d)
    panner_schedule, tt = pannerselvam(p, d)
    psk_schedule, tardiness__ = PSK(p, d)
    
    #tardiness
    temp_edd = final_tardiness(p,d,edd_schedule)
    temp_mdd_chal = final_tardiness(p, d, mddchallenger_schedule)
    temp_chal = final_tardiness(p, d, challenger4_schedule)
    temp_mdd = final_tardiness(p, d, mdd_schedule)
    temp_panner = final_tardiness(p, d, panner_schedule)
    
    #append to list
    tardiness_edd.append(temp_edd)
    tardiness_mdd_chal.append(temp_mdd_chal)
    tardiness_challenger.append(temp_chal)
    tardiness_mdd.append(temp_mdd)
    #tardiness_panners.append(tt)
    tardiness_panners.append(temp_panner)
    tardiness_psk.append(tardiness__)
chal_optgap = (sum(opt_soln)- sum(tardiness_challenger))/sum(opt_soln)
mddchal_optgap = (sum(opt_soln)- sum(tardiness_mdd_chal))/sum(opt_soln)
edd_optgap = (sum(opt_soln)- sum(tardiness_edd))/sum(opt_soln)
mdd_optgap = (sum(opt_soln)- sum(tardiness_mdd))/sum(opt_soln)
psk_optgap = (sum(opt_soln)- sum(tardiness_psk))/sum(opt_soln)
panner_optgap = (sum(opt_soln)- sum(tardiness_panners))/sum(opt_soln)

print('edd optimality gap for 0.2, 0.6: {}'.format(-edd_optgap))
print('edd challenger optimality gap for 0.2, 0.6: {}'.format(-chal_optgap))
print('mdd optimality gap for 0.2, 0.6: {}'.format(-mdd_optgap))
print('mdd challenger optimality gap for 0.2, 0.6: {}'.format(-mddchal_optgap))
print('psk optimality gap for 0.2, 0.6: {}'.format(-psk_optgap))
print('pannerselvam optimality gap for 0.2, 0.6: {}'.format(-panner_optgap))

print('optimal value for 0.2, 0.6: {}'.format(sum(opt_soln)/10))

edd optimality gap for 0.2, 0.6: 0.6681617934087072
edd challenger optimality gap for 0.2, 0.6: 0.5717338895330903
mdd optimality gap for 0.2, 0.6: 0.05302805720662739
mdd challenger optimality gap for 0.2, 0.6: 0.0521470165293395
psk optimality gap for 0.2, 0.6: 0.05302460343507185
pannerselvam optimality gap for 0.2, 0.6: 0.2672392745796904
optimal value for 0.2, 0.6: 1216061.9


In [58]:
#R = 0.2 T = 0.8
opt_soln = [] 
tardiness_edd = []
tardiness_challenger = []
tardiness_mdd_chal = []
tardiness_mdd = []
tardiness_panners = []
tardiness_psk = []
for i in range(30, 40):

    o = data[i]['optimal_solution']
    opt_soln.append(o)
    p = data[i]["processing_times"].copy()
    d = data[i]["due_dates"].copy()
    
    #function call
    edd_schedule = EDD(p, d)
    mddchallenger_schedule = MDD_Challenger(p, d)
    challenger4_schedule = EDD_Challenger(p, d)
    mdd_schedule = MDD(p, d)
    panner_schedule, tt = pannerselvam(p, d)
    psk_schedule, tardiness__ = PSK(p, d)
    
    #tardiness
    temp_edd = final_tardiness(p,d,edd_schedule)
    temp_mdd_chal = final_tardiness(p, d, mddchallenger_schedule)
    temp_chal = final_tardiness(p, d, challenger4_schedule)
    temp_mdd = final_tardiness(p, d, mdd_schedule)
    temp_panner = final_tardiness(p, d, panner_schedule)
    
    #append to list
    tardiness_edd.append(temp_edd)
    tardiness_mdd_chal.append(temp_mdd_chal)
    tardiness_challenger.append(temp_chal)
    tardiness_mdd.append(temp_mdd)
    tardiness_panners.append(temp_panner)
    tardiness_psk.append(tardiness__)
chal_optgap = (sum(opt_soln)- sum(tardiness_challenger))/sum(opt_soln)
mddchal_optgap = (sum(opt_soln)- sum(tardiness_mdd_chal))/sum(opt_soln)
edd_optgap = (sum(opt_soln)- sum(tardiness_edd))/sum(opt_soln)
mdd_optgap = (sum(opt_soln)- sum(tardiness_mdd))/sum(opt_soln)
psk_optgap = (sum(opt_soln)- sum(tardiness_psk))/sum(opt_soln)
panner_optgap = (sum(opt_soln)- sum(tardiness_panners))/sum(opt_soln)
print('edd optimality gap for 0.2, 0.8: {}'.format(-edd_optgap))
print('edd challenger optimality gap for 0.2, 0.8: {}'.format(-chal_optgap))
print('mdd optimality gap for 0.2, 0.8: {}'.format(-mdd_optgap))
print('mdd challenger optimality gap for 0.2, 0.8: {}'.format(-mddchal_optgap))
print('psk optimality gap for 0.2, 0.8: {}'.format(-psk_optgap))
print('pannerselvam optimality gap for 0.2, 0.8: {}'.format(-panner_optgap))

print('optimal value for 0.2, 0.8: {}'.format(sum(opt_soln)/10))

edd optimality gap for 0.2, 0.8: 0.651266622954235
edd challenger optimality gap for 0.2, 0.8: 0.5761654370719065
mdd optimality gap for 0.2, 0.8: 0.03201889520850162
mdd challenger optimality gap for 0.2, 0.8: 0.03122516947574145
psk optimality gap for 0.2, 0.8: 0.03201102305600279
pannerselvam optimality gap for 0.2, 0.8: 0.10738535135395279
optimal value for 0.2, 0.8: 2350056.1


In [59]:
#R = 0.6 T = 0.4
opt_soln = [] 
tardiness_edd = []
tardiness_challenger = []
tardiness_mdd_chal = []
tardiness_mdd = []
tardiness_panners = []
tardiness_psk = []
for i in range(90, 100):

    o = data[i]['optimal_solution']
    opt_soln.append(o)
    p = data[i]["processing_times"].copy()
    d = data[i]["due_dates"].copy()
    
    #function call
    edd_schedule = EDD(p, d)
    mddchallenger_schedule = MDD_Challenger(p, d)
    challenger4_schedule = EDD_Challenger(p, d)
    mdd_schedule = MDD(p, d)
    panner_schedule, tt = pannerselvam(p, d)
    psk_schedule, tardiness__ = PSK(p, d)
    
    #tardiness
    temp_edd = final_tardiness(p,d,edd_schedule)
    temp_mdd_chal = final_tardiness(p, d, mddchallenger_schedule)
    temp_chal = final_tardiness(p, d, challenger4_schedule)
    temp_mdd = final_tardiness(p, d, mdd_schedule)
    temp_panner = final_tardiness(p, d, panner_schedule)
    
    #append to list
    tardiness_edd.append(temp_edd)
    tardiness_mdd_chal.append(temp_mdd_chal)
    tardiness_challenger.append(temp_chal)
    tardiness_mdd.append(temp_mdd)
    tardiness_panners.append(temp_panner)
    tardiness_psk.append(tardiness__)
chal_optgap = (sum(opt_soln)- sum(tardiness_challenger))/sum(opt_soln)
mddchal_optgap = (sum(opt_soln)- sum(tardiness_mdd_chal))/sum(opt_soln)
edd_optgap = (sum(opt_soln)- sum(tardiness_edd))/sum(opt_soln)
mdd_optgap = (sum(opt_soln)- sum(tardiness_mdd))/sum(opt_soln)
psk_optgap = (sum(opt_soln)- sum(tardiness_psk))/sum(opt_soln)
panner_optgap = (sum(opt_soln)- sum(tardiness_panners))/sum(opt_soln)
print('edd optimality gap for 0.6, 0.4: {}'.format(-edd_optgap))
print('edd challenger optimality gap for 0.6, 0.4: {}'.format(-chal_optgap))
print('mdd optimality gap for 0.6, 0.4: {}'.format(-mdd_optgap))
print('mdd challenger optimality gap for 0.6, 0.4: {}'.format(-mddchal_optgap))
print('psk optimality gap for 0.6, 0.4: {}'.format(-psk_optgap))
print('pannerselvam optimality gap for 0.6, 0.4: {}'.format(-panner_optgap))

print('optimal value for 0.6, 0.4: {}'.format(sum(opt_soln)/10))

edd optimality gap for 0.6, 0.4: 0.7747722587648997
edd challenger optimality gap for 0.6, 0.4: 0.6040546252959849
mdd optimality gap for 0.6, 0.4: 0.012184957505327455
mdd challenger optimality gap for 0.6, 0.4: 0.011586196104682183
psk optimality gap for 0.6, 0.4: 0.012184957505327455
pannerselvam optimality gap for 0.6, 0.4: 4.188044766923159
optimal value for 0.6, 0.4: 92023.3


In [60]:
#R = 0.8 T = 0.8
opt_soln = [] 
tardiness_edd = []
tardiness_challenger = []
tardiness_mdd_chal = []
tardiness_mdd = []
tardiness_panners = []
tardiness_psk = []
for i in range(150, 160):
    o = data[i]['optimal_solution']
    opt_soln.append(o)
    p = data[i]["processing_times"].copy()
    d = data[i]["due_dates"].copy()
    
    #function call
    edd_schedule = EDD(p, d)
    mddchallenger_schedule = MDD_Challenger(p, d)
    challenger4_schedule = EDD_Challenger(p, d)
    mdd_schedule = MDD(p, d)
    panner_schedule, tt = pannerselvam(p, d)
    psk_schedule, tardiness__ = PSK(p, d)
    
    #tardiness
    temp_edd = final_tardiness(p,d,edd_schedule)
    temp_mdd_chal = final_tardiness(p, d, mddchallenger_schedule)
    temp_chal = final_tardiness(p, d, challenger4_schedule)
    temp_mdd = final_tardiness(p, d, mdd_schedule)
    temp_panner = final_tardiness(p, d, panner_schedule)
    
    #append to list
    tardiness_edd.append(temp_edd)
    tardiness_mdd_chal.append(temp_mdd_chal)
    tardiness_challenger.append(temp_chal)
    tardiness_mdd.append(temp_mdd)
    tardiness_panners.append(temp_panner)
    tardiness_psk.append(tardiness__)
chal_optgap = (sum(opt_soln)- sum(tardiness_challenger))/sum(opt_soln)
mddchal_optgap = (sum(opt_soln)- sum(tardiness_mdd_chal))/sum(opt_soln)
edd_optgap = (sum(opt_soln)- sum(tardiness_edd))/sum(opt_soln)
mdd_optgap = (sum(opt_soln)- sum(tardiness_mdd))/sum(opt_soln)
psk_optgap = (sum(opt_soln)- sum(tardiness_psk))/sum(opt_soln)
panner_optgap = (sum(opt_soln)- sum(tardiness_panners))/sum(opt_soln)
print('edd optimality gap for 0.8, 0.8: {}'.format(-edd_optgap))
print('edd challenger optimality gap for 0.8, 0.8: {}'.format(-chal_optgap))
print('mdd optimality gap for 0.8, 0.8: {}'.format(-mdd_optgap))
print('mdd challenger optimality gap for 0.8, 0.8: {}'.format(-mddchal_optgap))
print('psk optimality gap for 0.8, 0.8: {}'.format(-psk_optgap))
print('pannerselvam optimality gap for 0.8, 0.8: {}'.format(-panner_optgap))

print('optimal value for 0.8, 0.8: {}'.format(sum(opt_soln)/10))

edd optimality gap for 0.8, 0.8: 0.6966171484644914
edd challenger optimality gap for 0.8, 0.8: 0.6016573295360026
mdd optimality gap for 0.8, 0.8: 8.091373908960748e-05
mdd challenger optimality gap for 0.8, 0.8: 6.041753923598249e-05
psk optimality gap for 0.8, 0.8: 7.502290730269864e-05
pannerselvam optimality gap for 0.8, 0.8: 0.3100152952282851
optimal value for 0.8, 0.8: 2054039.3


In [ ]:
################################################################
#Comparison of Local Search Variations

In [71]:
import numpy as np
import os
#path = 'data/data_20_800.npy'
#path = 'data/data_100_200.npy'
path = 'data/data_200_200.npy'
#path = 'data/data_500_200.npy'
with open(path, 'rb') as f:
    data = np.load(f, allow_pickle=True)
len(data)

200

In [63]:
def Augmented_MDD_Challenger(processing_times: np.ndarray, due_dates: np.ndarray) -> np.ndarray:
    """ Find mathematical heuristic function skeleton for the single machine scheduling problem. Each job is assigned to the machine exactly once. Do not manipulate original due dates or processing times.

    Args:
        processing_times: A numpy array representing processing times of jobs.
        due_dates: A numpy array representing due dates of jobs.

    Return:
        A numpy array representing indices of jobs assigned in order as the result of applying the mathematical function to the inputs.
    """
    """Ultimate version of the job assignment function for the single machine scheduling problem."""
    current_time = 0
    schedule = []
    unscheduled_jobs = np.arange(len(processing_times))
    weighting = 1.1
    
    sort_order = np.lexsort((processing_times[unscheduled_jobs], due_dates[unscheduled_jobs]))
    unscheduled_jobs = unscheduled_jobs[sort_order]
    
    while unscheduled_jobs.size > 0:
        urgencies = np.maximum(processing_times[unscheduled_jobs] * 1.1 + current_time, due_dates[unscheduled_jobs])
        urgency_multiplier = (processing_times[unscheduled_jobs] / (current_time + np.max(processing_times[unscheduled_jobs])))
        urgency_multiplier[urgency_multiplier > 1] = 1
        urgency_bonus = (urgency_multiplier ** 2) / (1 + urgency_multiplier ** 2)
        urgencies *= (1 + urgency_bonus)
        impact_on_current_time = processing_times[unscheduled_jobs] / (current_time + np.sum(processing_times[unscheduled_jobs]) / len(unscheduled_jobs))
        urgencies += impact_on_current_time
        next_job = unscheduled_jobs[np.argmin(urgencies)]
        schedule.append(next_job)
        current_time += processing_times[next_job]
        unscheduled_jobs = np.delete(unscheduled_jobs, np.where(unscheduled_jobs == next_job))

        #search
        best_tardiness = calculate_tardiness(schedule, p, d)
        best_schedule = schedule
        for i in range(len(schedule) - 1):
            trial_schedule = schedule[:i] + schedule[i+1:] + [schedule[i]]
            trial_tardiness = calculate_tardiness(trial_schedule, p, d)
            if trial_tardiness < best_tardiness:
                best_tardiness = trial_tardiness
                best_schedule = trial_schedule[:]
        schedule = best_schedule
    
    return np.array(schedule)

In [64]:
def AUG_MDD(p, d):
    # Initialize current time
    current_time = 0

    # Create an array to track the scheduling order
    schedule = []

    # Number of jobs
    num_jobs = len(p)

    # Create an index array for tracking unscheduled jobs
    unscheduled_jobs = np.arange(num_jobs)

    # While there are unscheduled jobs
    while unscheduled_jobs.size > 0:
        # Calculate urgencies for each unscheduled job
        #urgencies = np.maximum(p[unscheduled_jobs], d[unscheduled_jobs] - current_time)
        urgencies = np.maximum(p[unscheduled_jobs]+ current_time, d[unscheduled_jobs] )
        # Find the job with the minimum urgency
        index_min_urgency = np.argmin(urgencies)
        job_to_schedule = unscheduled_jobs[index_min_urgency]
        
        # Append job id (index + 1 for 1-based id) to schedule
        schedule.append(job_to_schedule)
        
        # Update current time
        current_time += p[job_to_schedule]
        
        # Remove the job from the list of unscheduled jobs
        unscheduled_jobs = np.delete(unscheduled_jobs, index_min_urgency)

        best_tardiness = calculate_tardiness(schedule, p, d)
        best_schedule = schedule[:]
        for i in range(len(schedule) - 1):
            trial_schedule = schedule[:i] + schedule[i+1:] + [schedule[i]]
            trial_tardiness = calculate_tardiness(trial_schedule, p, d)
            if trial_tardiness < best_tardiness:
                best_tardiness = trial_tardiness
                best_schedule = trial_schedule[:]
        schedule = best_schedule
    return schedule

In [72]:
opt_soln = [] 
tardiness_mdd = []
tardiness_mdd_chal = []
tardiness_mddaug = []
tardiness_mddcaug = []


for i in range(len(data)):
    o = data[i]['optimal_solution']
    opt_soln.append(o)
    p = data[i]["processing_times"].copy()
    d = data[i]["due_dates"].copy()
    
    #function call
    mdd_schedule = MDD(p, d)
    mddchallenger_schedule = MDD_Challenger(p, d)
    augmddc_schedule = Augmented_MDD_Challenger(p, d)
    augmdd_schedule = AUG_MDD(p, d)
    
    #tardiness
    temp_mdd = final_tardiness(p, d, mdd_schedule)
    temp_mdd_chal = final_tardiness(p, d, mddchallenger_schedule)
    temp_augmddc = final_tardiness(p, d, augmddc_schedule)
    temp_augmdd = final_tardiness(p, d, augmdd_schedule)
    
    #append to list
    tardiness_mdd.append(temp_mdd)
    tardiness_mdd_chal.append(temp_mdd_chal)
    tardiness_mddcaug.append(temp_augmddc)
    tardiness_mddaug.append(temp_augmdd)

    
mdd_optgap = (sum(opt_soln)- sum(tardiness_mdd))/sum(opt_soln)
mddchal_optgap = (sum(opt_soln)- sum(tardiness_mdd_chal))/sum(opt_soln)
mddcaug_optgap = (sum(opt_soln)- sum(tardiness_mddcaug))/sum(opt_soln)
mddaug_optgap = (sum(opt_soln)- sum(tardiness_mddaug))/sum(opt_soln)
print('average mdd optimality gap: {}'.format(-mdd_optgap))
print('average augmented mdd optimality gap: {}'.format(-mddaug_optgap))
print('average mdd challenger optimality gap: {}'.format(-mddchal_optgap))
print('average aug mdd challenger optimality gap: {}'.format(-mddcaug_optgap))


average mdd optimality gap: 0.01335262323552794
average augmented mdd optimality gap: 0.009999267284939696
average mdd challenger optimality gap: 0.01283168670817354
average aug mdd challenger optimality gap: 0.009533862447360033
